<a href="https://colab.research.google.com/github/darengarciaH/AI-Question-Answering/blob/main/pipeline/sac_pipeline.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Soccer Analytics Coach (SAC) - Main Notebook

This notebook demonstrates how to use the SAC utilities for building a RAG-based soccer analytics assistant.

## 1. Setup & Imports

In [ ]:
# Install required packages
!pip install -U -q "google-generativeai>=0.7.2" # Install the Python SDK
!pip install google-ai-haystack
!pip install streamlit
!pip install pyngrok
!pip install "sentence-transformers>=4.1.0"
!pip install -U deepeval

In [ ]:
# Mount Google Drive (for Colab)
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# Add the directory containing sac_utils.py to Python path
import sys
import os

# Update this path to where you stored sac_utils.py in your Drive
# For example: '/content/drive/MyDrive/Colab Notebooks/'
UTILS_PATH = '/content/drive/MyDrive/'

if UTILS_PATH not in sys.path:
    sys.path.append(UTILS_PATH)

print(f"Added {UTILS_PATH} to Python path")
print(f"Current sys.path: {sys.path}")

Added /content/drive/MyDrive/ to Python path
Current sys.path: ['/content', '/env/python', '/usr/lib/python312.zip', '/usr/lib/python3.12', '/usr/lib/python3.12/lib-dynload', '', '/usr/local/lib/python3.12/dist-packages', '/usr/lib/python3/dist-packages', '/usr/local/lib/python3.12/dist-packages/IPython/extensions', '/root/.ipython', '/content/drive/MyDrive/']


In [ ]:
import pprint
from sac_utils import (
    # Data loading
    load_team_stats,
    load_player_stats,
    load_keeper_stats,
    load_formations,
    load_ground_truth,
    # Data cleaning
    rename_team_columns,
    rename_player_columns,
    rename_keeper_columns,
    # Document creation
    create_team_documents,
    create_player_documents,
    create_keeper_documents,
    create_formation_documents,
    create_ground_truth_documents,
    # Document store
    create_document_store,
    # Pipeline
    build_pipeline,
    run_pipeline,
    DEFAULT_TEMPLATE,
    # Evaluation
    evaluate_response
)

# For evaluation
from deepeval.metrics import FaithfulnessMetric, AnswerRelevancyMetric, ContextualRelevancyMetric
from deepeval.test_case import LLMTestCase
from deepeval.models import GeminiModel

print("Successfully imported all utilities!")

Successfully imported all utilities!


## 2. Configuration

In [ ]:
# Set your API key using Colab's secrets
from google.colab import userdata

GEMINI_API_KEY = userdata.get('GeminiSecret')
os.environ['GEMINI_API_KEY'] = GEMINI_API_KEY

# Set data directory (update this to where your CSV files are stored in Drive)
DATA_DIR = "/content/drive/MyDrive/"

print("Configuration complete!")
print(f"Data directory: {DATA_DIR}")

Configuration complete!
Data directory: /content/drive/MyDrive/


## 3. Load and Clean Data

In [ ]:
import glob
import pandas as pd

In [ ]:
# Load all team stats files
team_files = glob.glob(DATA_DIR + "team_stats_*.csv")
team_stats = pd.concat([pd.read_csv(f) for f in team_files], ignore_index=True)

# Load all player stats files
player_files = glob.glob(DATA_DIR + "player_stats_*.csv")
player_stats = pd.concat([pd.read_csv(f) for f in player_files], ignore_index=True)

# Load all keeper stats files
keeper_files = glob.glob(DATA_DIR + "keeper_stats_*.csv")
keeper_stats = pd.concat([pd.read_csv(f) for f in keeper_files], ignore_index=True)

# Load formations
formations = load_formations(DATA_DIR + "formations.csv")

print(f"Loaded {len(team_stats)} team records (all leagues)")
print(f"Loaded {len(player_stats)} player records (all leagues)")
print(f"Loaded {len(keeper_stats)} keeper records (all leagues)")
print(f"Loaded {len(formations)} formation records")

Loaded 1044 team records (all leagues)
Loaded 20297 player records (all leagues)
Loaded 1180 keeper records (all leagues)
Loaded 10 formation records


In [ ]:
# Rename columns to readable names
team_stats = rename_team_columns(team_stats)
player_stats = rename_player_columns(player_stats)
keeper_stats = rename_keeper_columns(keeper_stats)

# Preview
team_stats.head()

,Unnamed: 0,team_id,team_name,league_id,season_id,roster_size,matches_played,total_goals,total_assists,total_non_penalty_goals,...,second_yellow_cards,fouls_against,fouls_committed,offsides,penalties_won,penalties_conceded,own_goals,ball_recoveries,aerial_duels_won,aerial_duels_lost
0,0,0cdc4311,Augsburg,20,2025-2026,22,7,12,8,11,...,0,103,73,19,1,2,0,297,118,126
1,1,054efa67,Bayern Munich,20,2025-2026,20,7,27,20,23,...,0,76,82,14,2,0,0,317,48,52
2,2,add600ae,Dortmund,20,2025-2026,22,7,13,12,13,...,0,89,69,13,1,1,0,283,93,74
3,3,f0ac8ee6,Eint Frankfurt,20,2025-2026,22,7,19,15,18,...,0,72,72,6,1,1,0,270,88,102
4,4,a486e511,Freiburg,20,2025-2026,23,7,10,5,7,...,0,60,93,16,1,1,0,249,134,142


## 4. Create Documents

In [ ]:
# Convert to records (dictionaries)
team_records = team_stats.fillna("").to_dict(orient="records")
player_records = player_stats.fillna("").to_dict(orient="records")
keeper_records = keeper_stats.fillna("").to_dict(orient="records")

/tmp/ipython-input-2229249664.py:2: UserWarning: DataFrame columns are not unique, some columns will be omitted.
  team_records = team_stats.fillna("").to_dict(orient="records")


In [ ]:
# Create Haystack documents
team_docs = create_team_documents(team_records)
player_docs = create_player_documents(player_records)
keeper_docs = create_keeper_documents(keeper_records)

# Combine all documents
all_docs = team_docs + player_docs + keeper_docs

print(f"Created {len(team_docs)} team documents")
print(f"Created {len(player_docs)} player documents")
print(f"Created {len(keeper_docs)} keeper documents")
print(f"Total documents: {len(all_docs)}")

Created 1044 team documents
Created 20297 player documents
Created 1180 keeper documents
Total documents: 22521


## 5. Create Document Store with Embeddings

In [ ]:
# Create document store (this will take a moment to embed all documents)
document_store = create_document_store(
    all_docs,
    embedding_model="sentence-transformers/all-MiniLM-L6-v2"
)

print(f"Document store created with {document_store.count_documents()} documents")

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

Batches:   0%|          | 0/591 [00:00<?, ?it/s]

Document store created with 18893 documents


## 6. Build RAG Pipeline

In [ ]:
# Build the pipeline
pipe = build_pipeline(
    document_store=document_store,
    template=DEFAULT_TEMPLATE,
    api_key=GEMINI_API_KEY,
    embedding_model="sentence-transformers/all-MiniLM-L6-v2",
    top_k=5
)

print("Pipeline built successfully!")

Pipeline built successfully!


## 7. Query the System

In [ ]:
# Ask a question
query = "What can Arsenal improve on?"

result = run_pipeline(pipe, query)

print("\n" + "="*80)
print("QUERY:", query)
print("="*80)
print("\nANSWER:")
print(result["answer"])
print("\n" + "="*80)
print("\nRETRIEVED CONTEXTS:")
for i, ctx in enumerate(result["contexts"], 1):
    print(f"\n--- Context {i} ---")
    print(ctx[:300] + "...")

## 8. Evaluate with DeepEval (Optional)

In [ ]:
import pandas as pd
import time
from deepeval.models import GeminiModel

# Test questions
test_questions = ['How is Mohamed Salah performing this season compared to last season?',
 "What are Liverpool's main strengths and weaknesses in the 2024/25 season?",
 'Which players in the Premier League have the best passing accuracy this season?',
 'Compare Manchester City defensive performance between the 2023/24 and 2024/25 seasons.',
 'What are the top 3 teams in La Liga by expected goals (xG) this season?',
 'How many goals has Erling Haaland scored from fast breaks this season?',
 'Which teams in Serie A have the highest possession percentage?',
 "What is Barcelona's average shot-on-target percentage for the 2024/25 season?",
 "How does Arsenal's pass completion rate compare to Chelsea's this season?",
 'Which Bundesliga teams have committed the most errors leading to goals?',
 'What formation would you recommend for Arsenal against a team that plays high pressing?',
 'How can Manchester United improve their xG?',
 "What tactical adjustments should a team make when facing Liverpool's counter-attacking style?",
 'Is a 4-3-3 the best formation for Arsenal?',
 "How should a team defend Bayern Munich's high defensive line?",
 'What formation would work best for a team with strong wing-backs but weak central midfielders?',
 'How can a team exploit opponents who struggle with set pieces?',
 "What tactical approach would you recommend for a team facing Barcelona's midfield?",
 'When should a team switch from a 4-2-3-1 to a 3-5-2 formation during a match?',
 'How can a team improve their xG?']

# Generate responses
responses = []
for i, question in enumerate(test_questions):
    print(f"[{i+1}/{len(test_questions)}] Generating response...")
    result = run_pipeline(pipe, question, top_k=10)
    responses.append(result['answer'])
    time.sleep(5)

# Create DataFrame
df = pd.DataFrame({
    'Question': test_questions,
    'Response': responses
})

# Save to CSV for either DeepEval or Rubric evaluation
df.to_csv(DATA_DIR + "SAC_Test_Questions_Responses.csv", index=False)
print(f"✓ Saved {DATA_DIR}SAC_Test_Questions_Responses.csv")



[1/20] Generating response...


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

[2/20] Generating response...


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

[3/20] Generating response...


...

[19/20] Generating response...


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

[20/20] Generating response...


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

✓ Saved /content/drive/MyDrive/SAC_Test_Questions_Responses.csv


In [ ]:
import pandas as pd
from deepeval.models import GeminiModel

# Load the master file
rubric_df = pd.read_csv(DATA_DIR + "SAC_Test_Questions_Responses.csv")

# Setup eval model
eval_model = GeminiModel(model_name="gemini-2.5-pro", api_key=GEMINI_API_KEY)

# Get columns
questions = rubric_df['Question'].tolist()
responses = rubric_df['Response'].tolist()

# Initialize score lists
faithfulness_scores = []
answer_relevancy_scores = []
contextual_relevancy_scores = []

# Evaluate each pair
for i, (question, response) in enumerate(zip(questions, responses)):
    print(f"[{i+1}/{len(questions)}] Evaluating...")

    try:
        # Get contexts
        result = run_pipeline(pipe, question, top_k=10)

        # Evaluate using the function
        scores = evaluate_response(question, response, result["contexts"], eval_model)

        faithfulness_scores.append(scores['faithfulness'])
        answer_relevancy_scores.append(scores['answer_relevancy'])
        contextual_relevancy_scores.append(scores['contextual_relevancy'])

    except Exception as e:
        print(f"  Error: {e}")
        faithfulness_scores.append(None)
        answer_relevancy_scores.append(None)
        contextual_relevancy_scores.append(None)

# Add score columns
rubric_df['faithfulness_score'] = faithfulness_scores
rubric_df['answer_relevancy_score'] = answer_relevancy_scores
rubric_df['contextual_relevancy_score'] = contextual_relevancy_scores

# Save
rubric_df.to_csv(DATA_DIR + "SAC_DeepEval_Results.csv", index=False)
print(f"\n✓ Saved {DATA_DIR}SAC_DeepEval_Results.csv")

[1/20] Evaluating...


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Output()

Output()

Output()

...

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Output()

Output()

Output()


✓ Saved /content/drive/MyDrive/SAC_DeepEval_Results.csv


## 9. Evaluate using Ground Truth Dataset

In [ ]:
# Load ground truth tactical articles
ground_truth = load_ground_truth(DATA_DIR + "Ground_Truth.csv")
print(f"Loaded {len(ground_truth)} tactical articles")

Loaded 26 tactical articles


In [ ]:
# Convert ground truth to records
ground_truth_records = ground_truth.fillna("").to_dict(orient="records")

# Create ground truth documents
ground_truth_docs = create_ground_truth_documents(ground_truth_records)
print(f"Created {len(ground_truth_docs)} ground truth documents")

Created 23 ground truth documents


In [ ]:
len(ground_truth_docs)

23

In [ ]:
# Create ground truth document store
ground_truth_document_store = create_document_store(
    ground_truth_docs,
    embedding_model="sentence-transformers/all-MiniLM-L6-v2"
)

print(f"Ground truth document store: {ground_truth_document_store.count_documents()} documents")

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Ground truth document store: 23 documents


In [ ]:
# Build ground truth pipeline
gt_pipe = build_pipeline(
    document_store=ground_truth_document_store,
    template=DEFAULT_TEMPLATE,
    api_key=GEMINI_API_KEY,
    embedding_model="sentence-transformers/all-MiniLM-L6-v2",
    top_k=5
)

print("Ground truth pipeline built successfully!")

Ground truth pipeline built successfully!


In [ ]:
# Ground truth questions
gt_questions = [
    "What formation has proven most effective for counter attacking the 2024/25 Premier League season, and what team leads in metrics?",
    "How many goals from fast breaks have been scored in the 2024/25 season compared to the entire 2023/24 season?",
    "How did Barcelona's midfield overload create problems for Real Madrid's defensive structure, particularly affecting players positioning?",
    "What formation did Real Madrid use out of possession during the match?",
    "How does Bayern Munich's shot efficiency rating of +8.5 demonstrate their effectiveness compared to their expected goals?",
    "What percentage of possession does Bayern Munich average under Vincent Kompany?",
    "How did AC Milan use the strategy of \"baiting the press\" to create counter-attacking opportunities against Inter?",
    "What formation did Paulo Fonseca use for AC Milan in this derby match?",
    "What defensive formation did Atletico Madrid employ, and how did their full-backs' positioning contribute to shutting down Real Madrid's attack?",
    "Which team has won the most Serie A championships, and how many titles do they have?",
    "How many La Liga titles have Real Madrid and Barcelona won combined between 1929 and 2024?",
    "How do you beat a high press when building from the back?",
    "Why do teams shift from a 4-3-3 into a 3-2-5 in possession?",
    "How should a team defend against inverted fullbacks?",
    "When should you use a double pivot instead of a single pivot?",
    "How do you break down a compact 4-4-2 low block?",
    "What triggers should a team use to start a counter-press?",
    "When should a team switch from zonal pressing to man-oriented pressing?",
    "How do you create numerical superiority in midfield vs. a 4-2-3-1?",
    "Why do top teams use asymmetrical fullback systems?",
    "How should a team reorganize after receiving a red card?",
    "When should a coach shift to a back three mid-match?"
]


In [ ]:
import time

COMPARISON_TEMPLATE = """
Question: {question}

Response 1 (Stats-based): {stats_answer}

Response 2 (Ground Truth): {gt_answer}

Evaluate how well these two responses work together on a scale of 1-5:
1 = Completely contradictory (answers disagree)
2 = Mostly contradictory
3 = Neither contradictory nor complementary
4 = Mostly complementary (answers work well together)
5 = Fully complementary (answers perfectly complement each other)

Respond with ONLY a single number from 1 to 5.
"""

# Store results
results = []

# Loop through all questions
for i, query in enumerate(gt_questions, 1):
    print(f"Processing question {i}/{len(gt_questions)}...")

    # Get stats response
    stats_result = run_pipeline(pipe, query, top_k=10)
    time.sleep(10)

    # Get ground truth response
    gt_result = run_pipeline(gt_pipe, query, top_k=1)
    time.sleep(10)

    # Get answers
    stats_answer = stats_result["answer"]
    gt_answer = gt_result["answer"]

    # Create comparison prompt
    comparison_prompt = COMPARISON_TEMPLATE.format(
        question=query,
        stats_answer=stats_answer,
        gt_answer=gt_answer
    )

    # Use LLM to compare
    llm = pipe.get_component("llm")
    comparison_result = llm.run(parts=[comparison_prompt])
    score_text = comparison_result["replies"][0].strip()

    # Extract score
    try:
        score = int(score_text)
    except:
        # Try to find a number in the response
        import re
        numbers = re.findall(r'\d+', score_text)
        score = int(numbers[0]) if numbers else 3

    # Store results
    results.append({
        "question": query,
        "stats_answer": stats_answer,
        "gt_answer": gt_answer,
        "score": score
    })

    print(f"Score: {score}/5")

# Create DataFrame with results
import pandas as pd
results_df = pd.DataFrame(results)

# Save to CSV
results_df.to_csv(DATA_DIR + "ground_truth_results.csv", index=False)
print(f"\nResults saved to {DATA_DIR}ground_truth_results.csv")
print(f"Average score: {results_df['score'].mean():.2f}/5")

Processing question 1/22...


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Score: 3/5
Processing question 2/22...


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

...

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Score: 4/5
Processing question 22/22...


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Score: 4/5

Results saved to /content/drive/MyDrive/ground_truth_results.csv
Average score: 3.36/5


#